In [1]:
import pandas as pd
import numpy as np
from typing import Dict
import optuna

# --- All Project Imports ---
from AlgoTrade.Utils.DataManager import DataManager
from AlgoTrade.Config.BacktestConfig import BacktestConfig
from AlgoTrade.Config.Enums import TradingMode
from AlgoTrade.Strat.Class.BaseStrategy import BaseStrategy
from AlgoTrade.Factories.IndicatorFactory import IndicatorFactory
from AlgoTrade.Sizing.AtrBandsSizer import AtrBandsSizer
from AlgoTrade.Optimizer.StudyRunner import run_optimization
from AlgoTrade.Analysis.BacktestAnalysis import BacktestAnalysis

In [2]:
symbol_ltf = "TRX/USDT:USDT"
tframe_ltf = "1m"
symbol_htf = "TRX/USDT:USDT"
tframe_htf = "1h"
start_date = "2024-01-01"
dm = DataManager(name="bitget")

In [3]:
ltf_data = dm.from_local(symbol_ltf, tframe_ltf, start_date)
htf_data = dm.from_local(symbol_htf, tframe_htf, start_date)

Loading data from d:\ComputerScience\Trading\Quant2\data\bitget\TRX_USDT_USDT_1m.csv...
Load completed successfully. Data shape: (773918, 5)
Loading data from d:\ComputerScience\Trading\Quant2\data\bitget\TRX_USDT_USDT_1h.csv...
Load completed successfully. Data shape: (12756, 5)


In [4]:
htf_data_resampled = htf_data.reindex(ltf_data.index, method="ffill")
merged = ltf_data.join(htf_data_resampled, rsuffix="_htf").dropna()

In [5]:
merged_analysis = merged.copy()

In [6]:
merged_analysis["direction"] = np.where(merged_analysis["close"] > merged_analysis["close"].shift(1), "up", "down")

In [7]:
merged_analysis.iloc[:60]["direction"].value_counts()

direction
down    30
up      30
Name: count, dtype: int64

In [47]:
# Map direction to numeric values
direction_numeric = merged_analysis["direction"].map({"up": 1, "down": -1})

# One-hot style flags for up/down
is_up = (merged_analysis["direction"] == "up").astype(int)
is_down = (merged_analysis["direction"] == "down").astype(int)

# Rolling count of ups and downs
up_count = is_up.rolling(window=60, min_periods=1).sum()
down_count = is_down.rolling(window=60, min_periods=1).sum()

# Assign signal based on counts
merged_analysis["signal"] = np.select(
    [up_count > 45, down_count > 15], [1, 0], default=0
)

In [48]:
new_merged_analysis = merged_analysis.iloc[range(0, len(merged_analysis), 60)]

In [49]:
new_merged_analysis

,open,high,low,close,volume,open_htf,high_htf,low_htf,close_htf,volume_htf,direction,signal
date,,,,,,,,,,,,
2024-01-01 00:00:00,0.10761,0.10779,0.10761,0.10779,62162.0000,0.10761,0.10839,0.10761,0.10839,319312.0,down,0
2024-01-01 01:00:00,0.10839,0.10839,0.10839,0.10839,0.0000,0.10839,0.10839,0.10808,0.10822,1228250.0,down,0
2024-01-01 02:00:00,0.10822,0.10829,0.10822,0.10829,2192.0000,0.10822,0.10829,0.10768,0.10784,980225.0,up,0
2024-01-01 03:00:00,0.10784,0.10787,0.10783,0.10784,3898.0000,0.10784,0.10787,0.10710,0.10714,523090.0,down,0
2024-01-01 04:00:00,0.10714,0.10714,0.10710,0.10714,24998.0000,0.10714,0.10727,0.10635,0.10657,710078.0,down,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-21 06:00:00,0.27315,0.27327,0.27314,0.27327,3816.4839,0.27253,0.27327,0.27195,0.27212,264108.0,up,0
2025-06-21 07:00:00,0.27418,0.27418,0.27416,0.27418,2711.2163,0.27253,0.27327,0.27195,0.27212,264108.0,down,0
2025-06-21 08:00:00,0.27395,0.27407,0.27395,0.27401,4394.0328,0.27253,0.27327,0.27195,0.27212,264108.0,up,0


In [50]:
from AlgoTrade.Backtester.BacktestRunner import BacktestRunner

In [51]:
config = BacktestConfig(
    initial_balance=5000.0,
    leverage=15,
    trading_mode=TradingMode.CROSS,
    enable_slippage=True,
    base_slippage_bps=1.5,
    maker_fee_rate=0.0002,
    taker_fee_rate=0.0006,
    # Enable trailing stop
    enable_trailing_stop=True,
    breakeven_trigger_pct=0.01,
    breakeven_sl_pct=0.001,
    midpoint_trigger_pct=0.5,
    midpoint_tp_extension_pct=0.3,
    midpoint_sl_adjustment_pct=0.2,
    # Exit on signal change
    exit_on_signal_opposite=True,
)

In [52]:
runner = BacktestRunner(config=config, data=new_merged_analysis)

In [53]:
analysis = runner.run()
analysis.print_metrics()

Starting backtest...
Enhanced backtest finished.

--- Backtest Results ---
Period: [2024-01-01] -> [2025-06-21]
Initial Balance:        5,000.00
Final Balance:          3,947.79
Initial Equity:         5,000.00
Final Equity:           3,947.79
Final Buy&Hold Equity:  12,707.58
ROI (Equity):           -21.04%
Profit:                 0.00
Loss:                   1,052.21
Sharpe Ratio:           -0.55
Sortino Ratio:          -0.12
Calmar Ratio:           -0.44
Max Drawdown:           33.63%
Total Trades:           1
Total Good Trades:      0
Total Bad Trades:       1
Avg PnL Good Trades:    0.00%
Avg PnL Bad Trades:     -105.22%
Win Rate:               0.00%
Loss Rate:              100.00%
Profit Factor:          0.00


In [54]:
analysis.show_interactive_trade_explorer()